In [ ]:
# from datasets import load_dataset
# dataset = load_dataset("imagefolder", data_dir="/content/inaturalist_12K")
# dataset.push_to_hub("demoyolo/inaturalist-12k")


In [1]:
!pip install rustworkx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
dataset = load_dataset("demoyolo/inaturalist-12k")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/705 [00:00<?, ?B/s]

train-00000-of-00007.parquet:   0%|          | 0.00/475M [00:00<?, ?B/s]

train-00001-of-00007.parquet:   0%|          | 0.00/441M [00:00<?, ?B/s]

train-00002-of-00007.parquet:   0%|          | 0.00/362M [00:00<?, ?B/s]

...

train-00006-of-00007.parquet:   0%|          | 0.00/516M [00:00<?, ?B/s]

validation-00000-of-00002.parquet:   0%|          | 0.00/308M [00:00<?, ?B/s]

validation-00001-of-00002.parquet:   0%|          | 0.00/325M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9999 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [19]:
import torch
device = "cpu"
dtype = torch.float32

In [60]:
# @title Dataset Class
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from datasets import load_dataset
from PIL import Image
import io

class iNaturalistDataset(Dataset):
    class_names = {
      '0': 'Amphibia',
      '1': 'Animalia',
      '2': 'Arachnida',
      '3': 'Aves',
      '4': 'Fungi',
      '5': 'Insecta',
      '6': 'Mammalia',
      '7': 'Mollusca',
      '8': 'Plantae',
      '9': 'Reptilia'
  }
    def __init__(self, hf_dataset, transform=None, split='train'):
        self.dataset = hf_dataset[split] if split in hf_dataset else hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]

        image = item['image']
        if not isinstance(image, Image.Image):
            if isinstance(image, bytes):
                image = Image.open(io.BytesIO(image))
            else:

                image = Image.fromarray(image)
        if image.mode != 'RGB':
            image = image.convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = item['label'] #if 'label' in item else item.get('labels', 0)

        return image, label

# # Load the dataset
# dataset = load_dataset("demoyolo/inaturalist-12k")

# Define transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
# @title dataloaders
print("Available splits:", dataset.keys())
train_dataset = iNaturalistDataset(dataset, transform=train_transform, split='train')
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)


val_dataset = iNaturalistDataset(dataset, transform=val_transform, split='validation')
val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)
# print(f"Training samples: {len(train_loader.dataset)}")

# # Get a sample batch
# sample_batch = next(iter(train_loader))
# images, labels = sample_batch
# print(f"Batch shape: {images.shape}")
# print(f"Labels shape: {labels.shape}")
# print(f"Image dtype: {images.dtype}")
# print(f"Labels dtype: {labels.dtype}")
# try:
#     if hasattr(dataset[list(dataset.keys())[0]], 'features'):
#         features = dataset[list(dataset.keys())[0]].features
#         if 'label' in features:
#             class_names = features['label'].names
#             print(f"Number of classes: {len(class_names)}")
#             print(f"First few classes: {class_names[:5]}")
# except:
#     print("Class information not readily available")

Available splits: dict_keys(['train', 'validation'])


In [67]:
# @title calculation of input output size for the network
# Calculations for CNN Architecture
#The calculations assume an input image shape of 224x224 with 3 color channels (RGB).

# Layer 1: Convolutional Block 1
# Input: 224x224x3

# Conv2D Layer: (3, 3) filter, num_filters=32. The Keras Conv2D layer with default parameters uses a stride of 1, a padding of 0 (valid), and a dilation rate of 1.

# Output size:

# W_out = (224 - 3 + 2*0)/1 + 1 = 222

# H_out = (224 - 3 + 2*0)/1 + 1 = 222

# Output shape: 222x222x32

# MaxPool2D Layer: pool_size=(2, 2), strides=(2, 2).

# Output size:

# W_out = (222 - 2 + 2*0)/2 + 1 = 111

# H_out = (222 - 2 + 2*0)/2 + 1 = 111

# Output shape: 111x111x32

# Layer 2: Convolutional Block 2
# Input: 111x111x32

# Conv2D Layer: (3, 3) filter, num_filters=64. Stride 1, padding 0, dilation rate 1.

# Output size:

# W_out = (111 - 3 + 2*0)/1 + 1 = 109

# H_out = (111 - 3 + 2*0)/1 + 1 = 109

# Output shape: 109x109x64

# MaxPool2D Layer: pool_size=(2, 2), strides=(2, 2).

# Output size:

# W_out = (109 - 2 + 2*0)/2 + 1 = 54.5 -> floor(54.5) + 1 = 54

# H_out = (109 - 2 + 2*0)/2 + 1 = 54.5 -> floor(54.5) + 1 = 54

# Output shape: 54x54x64

# Layer 3: Convolutional Block 3
# Input: 54x54x64

# Conv2D Layer: (3, 3) filter, num_filters=128. Stride 1, padding 0, dilation rate 1.

# Output size:

# W_out = (54 - 3 + 2*0)/1 + 1 = 52

# H_out = (54 - 3 + 2*0)/1 + 1 = 52

# Output shape: 52x52x128

# MaxPool2D Layer: pool_size=(2, 2), strides=(2, 2).

# Output size:

# W_out = (52 - 2 + 2*0)/2 + 1 = 26

# H_out = (52 - 2 + 2*0)/2 + 1 = 26

# Output shape: 26x26x128

# Layer 4: Convolutional Block 4
# Input: 26x26x128

# Conv2D Layer: (5, 5) filter, num_filters=256. Stride 1, padding 0, dilation rate 1.

# Output size:

# W_out = (26 - 5 + 2*0)/1 + 1 = 22

# H_out = (26 - 5 + 2*0)/1 + 1 = 22

# Output shape: 22x22x256

# MaxPool2D Layer: pool_size=(2, 2), strides=(2, 2).

# Output size:

# W_out = (22 - 2 + 2*0)/2 + 1 = 11

# H_out = (22 - 2 + 2*0)/2 + 1 = 11

# Output shape: 11x11x256

# Layer 5: Convolutional Block 5
# Input: 11x11x256

# Conv2D Layer: (7, 7) filter, num_filters=512. Stride 1, padding 0, dilation rate 1.

# Output size:

# W_out = (11 - 7 + 2*0)/1 + 1 = 5

# H_out = (11 - 7 + 2*0)/1 + 1 = 5

# Output shape: 5x5x512

# MaxPool2D Layer: pool_size=(2, 2), strides=(2, 2).

# Output size:

# W_out = (5 - 2 + 2*0)/2 + 1 = 2.5 -> floor(2.5) + 1 = 2

# H_out = (5 - 2 + 2*0)/2 + 1 = 2.5 -> floor(2.5) + 1 = 2

# Output shape: 2x2x512

# Final Dense Layer
# Flatten Layer: The output of the last max-pooling layer is a 2x2x512 tensor. The flatten layer reshapes this into a one-dimensional vector.

# Vector size: 2 * 2 * 512 = 2048

# Dense Layer: The input to the first dense layer will be a vector of size 2048.

In [ ]:
class_names = {
    '0': 'Amphibia',
    '1': 'Animalia',
    '2': 'Arachnida',
    '3': 'Aves',
    '4': 'Fungi',
    '5': 'Insecta',
    '6': 'Mammalia',
    '7': 'Mollusca',
    '8': 'Plantae',
    '9': 'Reptilia'
}

print(class_names)

{'0': 'Amphibia', '1': 'Animalia', '2': 'Arachnida', '3': 'Aves', '4': 'Fungi', '5': 'Insecta', '6': 'Mammalia', '7': 'Mollusca', '8': 'Plantae', '9': 'Reptilia'}


In [75]:
import os

file_path = '/content/cnn_model.pt'
if os.path.exists(file_path):
    file_size_bytes = os.path.getsize(file_path)
    file_size_mb = file_size_bytes / (1024 * 1024)
    print(f"The size of the file at {file_path} is {file_size_bytes} bytes ({file_size_mb:.2f} MB).")
else:
    print(f"File not found at {file_path}")

The size of the file at /content/cnn_model.pt is 33590076 bytes (32.03 MB).
